In [3]:
import json
import os
import openai
client = openai.OpenAI()

In [28]:
a = {
    "section_name": "self-check-questions",
    "section_text": "Question: \n10. \n\nMany financial analysts and economists eagerly await the press releases for the reports on the home price index and consumer confidence index. What would be the effects of a negative report on both of these? What about a positive report?\n\n Answer: 10. \n\nA negative report on home prices would make consumers feel like the value of their homes, which for most Americans is a major portion of their wealth, has declined. A negative report on consumer confidence would make consumers feel pessimistic about the future. Both of these would likely reduce consumer spending, shifting AD to the left, reducing GDP and the price level. A positive report on the home price index or consumer confidence would do the opposite.\n\n",
    "chapter_name": "11-self-check-questions",
    "textbook_name": "principles-macroeconomics-3e",
    "chapter_url": "https://openstax.org/books/principles-macroeconomics-3e/pages/11-self-check-questions",
    "question": "\n10. \n\nMany financial analysts and economists eagerly await the press releases for the reports on the home price index and consumer confidence index. What would be the effects of a negative report on both of these? What about a positive report?\n\n",
    "answer": "10. \n\nA negative report on home prices would make consumers feel like the value of their homes, which for most Americans is a major portion of their wealth, has declined. A negative report on consumer confidence would make consumers feel pessimistic about the future. Both of these would likely reduce consumer spending, shifting AD to the left, reducing GDP and the price level. A positive report on the home price index or consumer confidence would do the opposite.\n\n"
}

In [4]:
econ_data = json.load(open('openstax_scraper/output_econ.json'))

# {'self-check-questions', 'keyterms', 'keyconceptsandsummary'}

In [13]:
self_check_questions = []
keyterms = []
keyconceptsandsummary = []

for item in econ_data:
    if item['section_name'] == 'self-check-questions':
        self_check_questions.append(item)
    elif item['section_name'] == 'keyterms':
        keyterms.append(item)
    elif item['section_name'] == 'keyconceptsandsummary':
        keyconceptsandsummary.append(item)

print(len(self_check_questions))
print(len(keyterms))
print(len(keyconceptsandsummary))


724
75
75


In [16]:
current_question = self_check_questions[0]

In [27]:
current_answer = {
    "section_name": "self-check-questions",
    "section_text": "Question: \n2. \n\n  What are the drawbacks to analyzing the global economy on a regional basis?\n  \n Answer: 2. \n\n  A region can have some of high-income countries and some of the low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning. For example, if you were to compare per capital real GDP for the United States, Canada, Haiti, and Honduras, it looks much different than if you looked at the same data for North America as a whole. Thus, regional comparisons are broad-based and may not adequately capture an individual country’s economic attributes.\n  \n",
    "chapter_name": "32-self-check-questions",
    "textbook_name": "principles-economics-3e",
    "chapter_url": "https://openstax.org/books/principles-economics-3e/pages/32-self-check-questions",
    "question": "\n2. \n\n  What are the drawbacks to analyzing the global economy on a regional basis?\n  \n",
    "answer": "2. \n\n  A region can have some of high-income countries and some of the low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning. For example, if you were to compare per capital real GDP for the United States, Canada, Haiti, and Honduras, it looks much different than if you looked at the same data for North America as a whole. Thus, regional comparisons are broad-based and may not adequately capture an individual country’s economic attributes.\n  \n"
}

In [28]:
prompt = f"""
Question: {current_answer['question']}
Gold Answer: {current_answer['answer']}
Analyze the question and answer, and find three important key points or concepts which is critical to this question and answer.
Based on the three key points, provide three incorrect answer choices that very likely confuse a student.
For each incorrect choice, only change one key point, and keep other parts unchanged.
Give reasoning for each incorrect choice. Think it step by step. "
Return a JSON object in this exact format:
        {{
            "Analysis": "Analysis of the question, answer, and the three key points",
            "question": "string",
            "choice": {{
                "gold_choice": {{"reason": "string","answer": "string"}},
                "incorrect_choice_1": {{"reason": "string, mention which key point is changed","answer": "string"}},
                "incorrect_choice_2": {{"reason": "string, mention which key point is changed","answer": "string"}},
                "incorrect_choice_3": {{"reason": "string, mention which key point is changed","answer": "string"}},
            }}
        }}
"""
temperature = 0.0

response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a knowledgeable teacher creating a multiple-choice question. Your task is to make the question challenging by generating three incorrect answer choices that might confuse a student."},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            response_format={"type": "json_object"}
        )

gpt_answer = response.choices[0].message.content


In [23]:
prompt = f"""
You are a knowledgeable teacher creating a challenging multiple-choice question with carefully crafted incorrect answers.

Step 1: Analyze the question and correct answer. Identify three critical concepts or key points essential for understanding and solving this question. These points will be used to generate incorrect answers.

Step 2: Based on these three key points, create three incorrect answer choices with the following requirements:
- Each incorrect choice should modify only **one** key point while keeping other parts consistent with the correct answer.
- Each incorrect answer should appear plausible to students who might misunderstand that particular key point.

For each incorrect answer, provide:
- The key point being modified.
- The incorrect answer choice.
- Reasoning to explain how the change in the key point could mislead students.

Return your output as a structured JSON object in the following format:

{{
    "Analysis": "Provide a thorough analysis of the question, the correct answer, and the three identified key points.",
    "question": "{current_answer['question']}",
    "choices": {{
        "gold_choice": {{
            "reason": "Explanation of why this is the correct answer",
            "answer": "{current_answer['answer']}"
        }},
        "incorrect_choices": [
            {{
                "key_point_changed": "Description of key point 1 being modified",
                "reason": "Explanation of why this incorrect answer is misleading and based on altering key point 1.",
                "answer": "Incorrect answer 1"
            }},
            {{
                "key_point_changed": "Description of key point 2 being modified",
                "reason": "Explanation of why this incorrect answer is misleading and based on altering key point 2.",
                "answer": "Incorrect answer 2"
            }},
            {{
                "key_point_changed": "Description of key point 3 being modified",
                "reason": "Explanation of why this incorrect answer is misleading and based on altering key point 3.",
                "answer": "Incorrect answer 3"
            }}
        ]
    }}
}}
"""

# API call for the structured response
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a knowledgeable teacher creating challenging multiple-choice questions with thoughtfully crafted incorrect answers."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.0,
    response_format={"type": "json_object"}
)

gpt_answer = response.choices[0].message.content

In [29]:
print(gpt_answer)

data = json.loads(gpt_answer)


{
    "Analysis": "The question addresses the drawbacks of analyzing the global economy on a regional basis. The gold answer highlights three key points: 1) the presence of both high-income and low-income countries within a region, 2) the variability of per capita real GDP across countries, and 3) the inadequacy of broad regional comparisons to capture individual country economic attributes. These points emphasize the complexity and limitations of regional economic analysis.",
    "question": "What are the drawbacks to analyzing the global economy on a regional basis?",
    "choice": {
        "gold_choice": {"reason": "This choice accurately reflects the complexities of regional economic analysis, including income disparities and the limitations of aggregated data.","answer": "A region can have some high-income countries and some low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning.

In [26]:
question = data["question"]
correct_answer = data["choices"]["gold_choice"]["answer"]
incorrect_answers = [{"label": f"incorrect_choice_{i+1}", "answer": choice["answer"]}
                     for i, choice in enumerate(data["choices"]["incorrect_choices"])]

# Display results
print("Question:", question)
print("Correct Answer:", correct_answer)
print("Incorrect Answers:")
for answer in incorrect_answers:
    print(answer["label"] + ":", answer["answer"])


Question: What are the drawbacks to analyzing the global economy on a regional basis?
Correct Answer: A region can have some high-income countries and some low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning. For example, if you were to compare per capita real GDP for the United States, Canada, Haiti, and Honduras, it looks much different than if you looked at the same data for North America as a whole. Thus, regional comparisons are broad-based and may not adequately capture an individual country’s economic attributes.
Incorrect Answers:
incorrect_choice_1: A region typically consists of countries with similar income levels, making regional analysis a reliable method for understanding economic conditions.
incorrect_choice_2: Aggregating per capita real GDP for a region provides a clear picture of the economic status of that region, as it smooths out individual country discrepancies

In [14]:
system_prompt_1 = """
You are a helpful assistant that can answer questions and help with tasks.
"""
prompt_1 = f"""
Question: {a['question']}
Please break down the question into smaller parts. 
And replace one part at a time, and then merge them(other parts unchanged) into a new questions that different from the original question.
"""
temperature = 0.0

response_1 = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt_1},
                {"role": "user", "content": prompt_1}
            ],
            temperature=temperature,
            # response_format={"type": "json_object"}
        )

gpt_answer_1 = response_1.choices[0].message.content


In [18]:
print(a['question'])
print(a['answer'])
print("--------------------------------")
print(gpt_answer_1)



2. 

Identify whether the market supply curve will shift right or left or will stay the same for the following:
Firms in an industry are required to pay a fine for their carbon dioxide emissions.
Companies are sued for polluting the water in a river.
Power plants in a specific city are not required to address the impact of their air quality emissions. 
Companies that use fracking to remove oil and gas from rock are required to clean up the damage.



2. 


supply shifts left
supply shifts left
 supply stays the same
supply shifts left



--------------------------------
Sure! Let's break down the original question into smaller parts and then replace one part at a time to create new questions.

### Original Parts:
1. Firms in an industry are required to pay a fine for their carbon dioxide emissions.
2. Companies are sued for polluting the water in a river.
3. Power plants in a specific city are not required to address the impact of their air quality emissions.
4. Companies that use fra

## Key terms